## **Dataset**

In [1]:
from datasets import load_dataset, concatenate_datasets

ds = load_dataset("uitnlp/vietnamese_students_feedback")

In [2]:
# Concatenate all splits into a single dataset
ds = concatenate_datasets([ds["train"], ds["test"], ds["validation"]])

In [3]:
ds=ds.remove_columns(["sentiment", "topic"])

In [4]:
import re

def clean_sentence(example):
    # Xóa khoảng cách trước dấu câu
    example['sentence'] = re.sub(r'\s+([.,!?;])', r'\1', example['sentence'])
    example['sentence'] = example['sentence'] + " "
    return example

ds = ds.map(clean_sentence)

In [5]:
ds[4]['sentence']

'thầy giảng bài hay, có nhiều bài tập ví dụ ngay trên lớp. '

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "NlpHUST/gpt2-vietnamese"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
len(tokenizer)

50258

In [8]:
tokenizer.pad_token = "<pad>"
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

In [9]:
tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.bos_token_id

(1, 50257, 50257)

In [10]:
def tokenize(example):
    return tokenizer(example["sentence"])

tokenized_ds = ds.map(
    tokenize, remove_columns=["sentence"], batched=True
)

In [11]:
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 16175
})

In [12]:
block_size = 128

def group_texts(examples):
    # concat input_ids
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    total_length = (total_length // block_size) * block_size

    # split block_size
    result = {
        k: [concatenated[k][i : i + block_size] for i in range(0, total_length, block_size)]
        for k in concatenated
    }

    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result

lm_ds = tokenized_ds.map(group_texts, batched=True)


In [13]:
lm_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1941
})

In [14]:
# split train/test
lm_ds = lm_ds.train_test_split(test_size=0.1)

In [15]:
lm_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 195
    })
})

## **Training**

In [16]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

training_args = TrainingArguments(
    output_dir="gpt2-small-vietnamese-students-feedback",
    logging_dir="logs",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=7e-5,
    num_train_epochs=30,
    eval_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    eval_steps=50,
    save_steps=50,
    logging_steps=50,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,
    bf16=True,
    gradient_checkpointing=False,
    gradient_accumulation_steps=2,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_ds["train"],
    eval_dataset=lm_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_29604\2984512798.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## **Training**

In [17]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,2.915400,2.709089
100,2.588200,2.614721
150,2.483100,2.579675
200,2.419300,2.562612
250,2.369700,2.554899
300,2.330900,2.551772
350,2.307300,2.551575
400,2.289700,2.551540


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=420, training_loss=2.4547282355172295, metrics={'train_runtime': 272.1543, 'train_samples_per_second': 192.464, 'train_steps_per_second': 1.543, 'total_flos': 3421619159040000.0, 'train_loss': 2.4547282355172295, 'epoch': 30.0})

In [18]:
import torch
torch.cuda.empty_cache()

In [19]:
# push to hub
trainer.push_to_hub("binhphap5/gpt2-small-vietnamese-students-feedback")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/binhphap5/gpt2-small-vietnamese-students-feedback/commit/7a9cbdd7e58706b68a2c1d2d95838b4d8a2fa06c', commit_message='binhphap5/gpt2-small-vietnamese-students-feedback', commit_description='', oid='7a9cbdd7e58706b68a2c1d2d95838b4d8a2fa06c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/binhphap5/gpt2-small-vietnamese-students-feedback', endpoint='https://huggingface.co', repo_type='model', repo_id='binhphap5/gpt2-small-vietnamese-students-feedback'), pr_revision=None, pr_num=None)

## **Inference**

In [20]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "binhphap5/gpt-small-c4"

# model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [26]:
prompt = "slide"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [27]:
output = model.generate(
    **inputs,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
)

In [28]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

slide quá khó hiểu, em đã học gần hết các phần trong sách rồi mà vẫn không hiểu, mong thầy có thể giải thích thêm các nội dung này. môn học nên kết hợp với thực hành ở nhà, thực hành trên máy để học lý thuyết, và thực hành trên lớp, thực hành trên máy nhiều khi còn nhàm chán. thầy dạy rất nhiệt tình và tâm huyết. thầy dạy rất nhiệt tình và tâm huyết, truyền đạt rất nhiều kinh nghiệm. cô dạy nhiệt tình, vui tính. giáo viên có thể chia sẻ nhiều kinh nghiệm dạy và học. cô giáo dễ thương, dạy dễ hiểu và


In [24]:
import math
import torch
# Shift for labels (causal LM setting: predict token t+1 from token t)
labels = output[:, 1:].clone()
inputs = output[:, :-1].clone()

with torch.no_grad():
    outputs = model(inputs)
    logits = outputs.logits

# Compute log softmax over vocabulary
log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
# Gather log-probabilities corresponding to the labels
selected_log_probs = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)

# Sum negative log probs → total NLL
nll = -selected_log_probs.sum().item()
num_tokens = labels.numel()
perplexity = math.exp(nll / num_tokens)
perplexity

4.699362770034395